<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 01 - Data collecting 📌
</div>

## I. Import libraries

In [ ]:
!pip install requests
!pip install pandas
!pip install beautifulsoup4
!pip install selenium
!pip install webdriver-manager

In [2]:
import os
import tempfile  # Make sure to import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager # Added import

## II. Config environment

In [3]:
CONFIG = {
    'url': 'https://topdev.vn/viec-lam-it',
    'login_url': 'https://accounts.topdev.vn/login/github',
    'login_github': 'https://github.com/login',
    'email': 'truongvietcong672@gmail.com',
    'password': 'Tienanh@12345',
    'max_pages': 1,
    'user_agents': [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    ],
    'timeout': 650,
    'retry_attempts': 3,
    'retry_delay': 5,
}

## III. Collecting link html

### a. Set up drive

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless=new') 
options.add_argument("window-size=1920,1080")

# User-agent ngẫu nhiên
user_agent = random.choice(CONFIG['user_agents'])
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)

### b. Load page

In [9]:
# Login Github
driver.get(CONFIG['login_github'])
wait = WebDriverWait(driver, 3)

wait.until(EC.element_to_be_clickable((By.NAME, "login"))).send_keys(CONFIG['email'])
wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(CONFIG['password'])
wait.until(EC.element_to_be_clickable((By.NAME, "commit"))).click()

driver.get(CONFIG['login_url'])
print("Verifying login status on TopDev...")
time.sleep(3) # Give a moment for the page to settle after redirect

try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'logout') or contains(@href, 'dang-xuat')] | //button[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'đăng xuất')] | //*[@aria-label='profile' or @aria-label='Profile' or @data-test-id='user-profile' or contains(@class, 'avatar') or contains(@class, 'profile')]"))
    )
    print("Login successful: Found an element indicating a logged-in state.")
except TimeoutException:
    print("Login verification failed: Could not find a clear indicator of a logged-in state within the timeout.")
    print(f"Current URL: {driver.current_url}")


driver.get(CONFIG['url'])

try:
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
except Exception as e:
    print(f"An error occurred during the crawling process: {e}")

    # Get total records
num_record = int(
    driver.find_element(
        By.CSS_SELECTOR, "h1.text-xl.font-bold.text-black span.text-primary"
    ).text
)
print(f"Total records to fetch: {num_record}")

# Click to VIETNAM
driver.find_element(
    By.XPATH,
    "//button[contains(@class, 'uppercase') and contains(text(), 'Vi')]"
).click()

timeout = CONFIG['timeout']
start_time = time.time()
temp = 0

# Crolling to the bottom of the page
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    current_num_record = len(driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0"))
    print(f"Current: {current_num_record}", end="\r")

    if current_num_record < temp:
        current_num_record = temp
        start_time = time.time()

    if current_num_record >= num_record:
        break

    if time.time() - start_time > timeout:
        print("Timeout: No change detected within the specified time.")
        break

    # temp = current_num_record

Verifying login status on TopDev...
Login successful: Found an element indicating a logged-in state.
Total records to fetch: 983


In [6]:
driver.quit()

### c. Get job url

In [ ]:
url_list = []
elements = driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0")

for i in range(len(elements)):
    try:
        # Refetch từng phần tử để tránh stale
        element = driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0")[i]
        a_tag = element.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        if href:
            url_list.append(href)
    except (StaleElementReferenceException, NoSuchElementException):
        print(f"Bỏ qua phần tử lỗi tại index {i}")
        continue
now = pd.to_datetime("now")
print(f"\nCrawling completed at {now}. Total URLs collected: {len(url_list)}")

for i, url in enumerate(url_list):
    print(f"{i + 1}/{len(url_list)}: {url}")

# Save URLs to CSV
df = pd.DataFrame(url_list, columns=["url"])
df.to_csv('../data/job_url.csv', index=False)


Bỏ qua phần tử lỗi tại index 5

Crawling completed at 2025-05-13 17:00:33.748435. Total URLs collected: 984
1/984: https://topdev.vn/viec-lam/ux-ui-designer-game-cong-ty-co-phan-suntek-2036351?src=topdev_search&medium=searchresult
2/984: https://topdev.vn/viec-lam/senior-devops-engineer-cong-ty-co-phan-cong-nghe-ung-dung-xanh-viet-nam-2036350?src=topdev_search&medium=searchresult
3/984: https://topdev.vn/viec-lam/ky-su-phan-mem-c-developer-urgent-aram-vina-2036349?src=topdev_search&medium=searchresult
4/984: https://topdev.vn/viec-lam/chuyen-vien-tham-dinh-mua-sam-cntt-ban-cong-nghe-bidv-2036342?src=topdev_search&medium=searchresult
5/984: https://topdev.vn/viec-lam/chuyen-vien-quan-ly-chuyen-doi-so-ban-cong-nghe-bidv-2036343?src=topdev_search&medium=searchresult
6/984: https://topdev.vn/viec-lam/chuyen-vien-cloud-ban-cong-nghe-bidv-2036341?src=topdev_search&medium=searchresult
7/984: https://topdev.vn/viec-lam/chuyen-vien-kien-truc-cntt-ban-cong-nghe-bidv-2036340?src=topdev_search&med

## IV. Crawl data

Ta sẽ chọn ra các trường thông tin sau:
- `job_title`: tên công việc (VD: IT Helpdesk, Data engineer, ...vv)

- `company_name`: tên công ty (VD: Vingroup, Momo, ...vv)

- `salary`: mức lương (VD: 2.000.000vnđ, 15.000.000vnđ..vv)

- `address`: địa chỉ (VD: Tân Bình, Bình thạnh...vv)

- `posted_date`: ngày đăng (VD: 21/6/2025, 21/7/2025)

- `industry`: ngành nghề (VD: Giáo dục, Phần mềm, ...vv)

- `company_size`: quy mô công ty (VD: 500-999 Nhân viên)

- `company_nationality`: quốc tịch công ty (VD: China, India, United-Kingdom, Vietnam)

- `years_experience`: năm kinh nghiệm (VD: từ 2 năm, ...vv)

- `position_level`: cấp bậc (VD: Intern, Middle, Senior)

- `employment_type`: loại hình (VD: In Office, Remote..vv)

- `contract_type`: loại hợp đồng (VD: Fulltime, Freelance)

- `technologies_used`: công nghệ sử dụng (VD: .NET, Java)

- `job_description`: mô tả công việc (VD: Trách nhiệm công việc là Làm việc với các bộ ph.)

- `company_url`: url thông tin công ty.

- `url`: url công việc

In [11]:
data = []

for i, url in enumerate(url_list):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    time.sleep(2)

    # Tên công việc
    job_title_ele = driver.find_elements(
        By.CSS_SELECTOR, 
        "h1.text-2xl.font-bold.text-black"
    )
    job_title = job_title_ele[0].text.strip() if job_title_ele else None

    # Tên công ty
    company_name_ele = driver.find_elements(
        By.CSS_SELECTOR, 
        "p.text-base.font-bold"
    )
    company_name = company_name_ele[0].text.strip() if company_name_ele else None

    # Mức lương
    salary_elem = driver.find_elements(
        By.CSS_SELECTOR, 
        "div.flex.items-center p.text-primary"
    )
    salary = salary_elem[0].text.strip() if salary_elem else None

    # Địa chỉ
    address_elem = driver.find_elements(
        By.CSS_SELECTOR, 
        "div[data-testid='flowbite-tooltip-target'] span"
    )
    address = [elem.text.strip() for elem in address_elem if elem.text.strip()]

    # Ngày đăng
    date_posted_elem = driver.find_elements(
        By.XPATH, 
        "//div[contains(text(), 'Đăng')]"
    )
    date_posted = [elem.text.split("và")[0].strip() for elem in date_posted_elem if elem.text.startswith("Đăng")]

    # Ngành nghề
    industry_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Ngành nghề']/following-sibling::p"
    )
    industry = [elem.text.strip() for elem in industry_elem if elem.text.strip()]

    # Quy mô công ty
    company_size_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Quy mô công ty']/following-sibling::p"
    )
    company_size = [elem.text.strip() for elem in company_size_elem if elem.text.strip()]

    # Quốc tịch công ty
    company_nationality_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Quốc tịch công ty']/following-sibling::p"
    )
    company_nationality = [elem.text.strip() for elem in company_nationality_elem if elem.text.strip()]

    # Năm kinh nghiệm
    experience_years_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Năm kinh nghiệm tối thiểu']/following-sibling::div//a"
    )
    experience_years = [elem.text.strip() for elem in experience_years_elem if elem.text.strip()]

    # Cấp bậc
    position_level_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Cấp bậc']/following-sibling::div//a"
    )
    position_level = [elem.text.strip() for elem in position_level_elem if elem.text.strip()]

    # Loại hình
    employment_type_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Loại hình']/following-sibling::div//a"
    )
    employment_type = [elem.text.strip() for elem in employment_type_elem if elem.text.strip()]

    # Hợp đồng
    contract_type_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Loại hợp đồng']/following-sibling::div//a"
    )
    contract_type = [elem.text.strip() for elem in contract_type_elem if elem.text.strip()]

    # Công nghệ sử dụng
    technologies_used_elem = driver.find_elements(
        By.XPATH, 
        "//div[contains(@class, 'flex flex-wrap')]//a/span"
    )
    technologies_used = [elem.text.strip() for elem in technologies_used_elem if elem.text.strip() and elem.text.strip() != 'Xem công ty']

    # Mô tả công việc
    job_description_elem = driver.find_elements(
        By.ID, 
        "JobDescription"
    )
    job_description = [elem.text.strip() for elem in job_description_elem if elem.text.strip()]

    # URL Công ty
    company_url_elem = driver.find_elements(
        By.XPATH, 
        "//a[contains(@class, 'break-all') and contains(@href, 'topdev.vn/nha-tuyen-dung')]"
    )
    company_url = [elem.get_attribute("href").strip() for elem in company_url_elem if elem.get_attribute("href")]

    # Lưu trữ dữ liệu
    job = [
        job_title,
        company_name,
        salary,
        address,
        date_posted,
        industry,
        company_size,
        company_nationality,
        experience_years,
        position_level,
        employment_type,
        contract_type,
        technologies_used,
        job_description,
        company_url,
        url
    ]
    data.append(job)

    print(f"Data was collected from {i + 1}/{len(url_list)}")

print("Data collection completed.")
print(f"Total records collected: {len(data)}")

Data was collected from 1/984
Data was collected from 2/984
Data was collected from 3/984
Data was collected from 4/984
Data was collected from 5/984
Data was collected from 6/984
Data was collected from 7/984
Data was collected from 8/984
Data was collected from 9/984
Data was collected from 10/984
Data was collected from 11/984
Data was collected from 12/984
Data was collected from 13/984
Data was collected from 14/984
Data was collected from 15/984
Data was collected from 16/984
Data was collected from 17/984
Data was collected from 18/984
Data was collected from 19/984
Data was collected from 20/984
Data was collected from 21/984
Data was collected from 22/984
Data was collected from 23/984
Data was collected from 24/984
Data was collected from 25/984
Data was collected from 26/984
Data was collected from 27/984
Data was collected from 28/984
Data was collected from 29/984
Data was collected from 30/984
Data was collected from 31/984
Data was collected from 32/984
Data was collecte

In [ ]:
collumns = [
    "job_title",
    "company_name",
    "salary",
    "address",
    "date_posted",
    "industry",
    "company_size",
    "company_nationality",
    "experience_years",
    "position_level",
    "employment_type",
    "contract_type",
    "technologies_used",
    "job_description",
    "company_url",
    "job_url"
]

df = pd.DataFrame(data, columns=collumns)

driver.quit()
df.head()

,job_title,company_name,salary,address,date_posted,industry,company_size,company_nationality,experience_years,position_level,employment_type,contract_type,technologies_used,job_description,company_url,job_url
0,UX/UI Designer (Game),CÔNG TY CỔ PHẦN SUNTEK,Thương lượng,"[Thành phố Thủ Đức, Hồ Chí Minh]",[Đăng 1 giờ trước],[Giải trí/ Game],[25-99 Nhân viên],[Thailand],[Từ 2 năm],"[Junior, Middle]",[In Office],[Fulltime],"[UX/UI Design, HTML & CSS, UI/UX]",[Trách nhiệm công việc\n1. General task\nHave ...,[https://topdev.vn/nha-tuyen-dung/cong-ty-co-p...,https://topdev.vn/viec-lam/ux-ui-designer-game...
1,Senior DevOps Engineer,CÔNG TY CỔ PHẦN CÔNG NGHỆ ỨNG DỤNG XANH VIỆT NAM,40.000.000 VND to 50.000.000 VND,"[Quận Ba Đình, Hà Nội]",[Đăng 3 giờ trước],[],[Nhân viên],[],[Từ 5 năm],[Senior],[In Office],[Fulltime],"[DevOps, Docker, Ansible, CI/CD, Terraform]","[GA là một công ty khởi nghiệp, phát triển và ...",[https://topdev.vn/nha-tuyen-dung/cong-ty-co-p...,https://topdev.vn/viec-lam/senior-devops-engin...
2,KỸ SƯ PHẦN MỀM (C# Developer) - Urgent,Aram Vina,Lên tới 1.500 VND,"[Quận Tây Hồ, Hà Nội]",[Đăng 5 giờ trước],"[Thiết Kế, Software, Gia công phần mềm, Triển ...",[25-99 Nhân viên],[South-Korea],[Không yêu cầu],"[Fresher, Junior, Middle]",[In Office],[Fulltime],"[C#, WPF, Software Developer]",[CÔNG TY TNHH ARAM VINA cần tuyển dụng GẤP KỸ ...,[https://topdev.vn/nha-tuyen-dung/aram-vina-93...,https://topdev.vn/viec-lam/ky-su-phan-mem-c-de...
3,Chuyên viên Thẩm định mua sắm CNTT,Ban Công nghệ BIDV,Thương lượng,"[Quận Hai Bà Trưng, Hà Nội]",[Đăng 5 giờ trước],[Ngân Hàng],[Hơn 1000 Nhân viên],[Vietnam],[Từ 3 năm],[Middle],[In Office],[Fulltime],"[IT Support, IT Service]",[Trách nhiệm công việc\nThực hiện công tác thẩ...,[https://topdev.vn/nha-tuyen-dung/ban-cong-ngh...,https://topdev.vn/viec-lam/chuyen-vien-tham-di...
4,Chuyên viên Quản lý Chuyển đổi số,Ban Công nghệ BIDV,Thương lượng,"[Quận Hai Bà Trưng, Hà Nội]",[Đăng 5 giờ trước],[Ngân Hàng],[Hơn 1000 Nhân viên],[Vietnam],[Từ 2 năm],"[Junior, Middle]",[In Office],[Fulltime],"[Agile, Project Manager, Product Owner, Busine...",[Trách nhiệm công việc\nPhối hợp triển khai th...,[https://topdev.vn/nha-tuyen-dung/ban-cong-ngh...,https://topdev.vn/viec-lam/chuyen-vien-quan-ly...


### V. Save data

In [14]:
df.to_csv('../data/raw_data.csv', index=False)
print("Data saved to data.csv")

Data saved to data.csv
